In [1]:
import numpy as np
import pandas as pd
import glob
import os
import googlemaps
import geocoder
import folium
import matplotlib.pyplot as plt
import geojsoncontour

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from folium.plugins import MarkerCluster
from scipy.interpolate import griddata

In [2]:
df = pd.read_csv('9-27-9-29_trip_latlng.csv')

In [3]:
df.head()

Unnamed: 0   Device   DeviceGroup           StartDate  Duration  \
0           0  FO15921  Hospital ITS  September 27, 2022         6   
1           1  FO15921  Hospital ITS  September 27, 2022         4   
2           2  FO15921  Hospital ITS  September 27, 2022         4   
3           3  FO15921  Hospital ITS  September 27, 2022        10   
4           4  FO15921  Hospital ITS  September 27, 2022         8   

                    EndDate  Distance StopDuration  \
0   Sep 27, 2022 8:42:12 AM         3         0:07   
1   Sep 27, 2022 8:53:34 AM         1         1:53   
2  Sep 27, 2022 10:51:02 AM         2         0:03   
3  Sep 27, 2022 11:05:09 AM         3         1:04   
4  Sep 27, 2022 12:18:46 PM         2         0:02   

                                      Location StopZoneType  IdleDuration  \
0  1690 S 900 W, Salt Lake City, UT 84104, USA          NaN             0   
1   1525 2100 S, Salt Lake City, UT 84119, USA          NaN             0   
2  1690 S 900 W, Salt Lake City, UT 84104, USA          NaN             0   
3    75 W 900 S, Salt Lake City, UT 84101, USA          NaN             0   
4    780 1300 E, Salt Lake City, UT 84102, USA          NaN             0   

   MaxSpeed        lat        long  
0        68  40.733824 -111.917589  
1        42  40.725374 -111.933127  
2        45  40.733824 -111.917589  
3        35  40.749723 -111.893200  
4        39  40.752255 -111.854501

In [4]:
SLC_COORDINATES = [40.7608, -111.891]

slc = folium.Map(location=SLC_COORDINATES, zoom_start=12)
marker_cluster = MarkerCluster().add_to(slc)

for index, row in df.iterrows():
    location = [row["lat"],row["long"]]
    folium.Marker(location).add_to(marker_cluster)

    # slc.Marker(
    #     location = [row["lat"],row["long"]],
    #     clustered_marker = True)

slc.save('stops.html')

In [ ]:
df_c = df[df.lat > 40]

y = df_c['lat']
x = df_c['long']
z = df_c['IdleDuration']

xi = np.linspace(x.min(),x.max(),500);
yi = np.linspace(y.min(),y.max(),500);
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')

#alphas[:, 30:] = np.linspace(1, 0, 70)
countour = plt.contourf(xi,yi,zi,cmap='Reds', alpha=0.3)

In [ ]:
geojson = geojsoncontour.contourf_to_geojson(
    contourf=countour,
    ndigits=3,
   )

In [ ]:
with open('countour_geojson.txt', 'w') as f:
    f.write(geojson)

In [ ]:
# Convert text file to JSON 
import json 
with open('countour_geojson.txt') as json_file:     # name of the text file
    data = json.load(json_file)

# Convert JSON to GeoJson 
from geojson import dump
with open('countour_geojson.geojson', 'w') as f:   # name of the output geojson file
   dump(data, f)